## Analyze Farm City User Review 


In [41]:
from googletrans import Translator
import pandas as pd 
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iKame_LinhNK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [57]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iKame_LinhNK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
df_5=pd.read_excel(r"C:\Users\iKame_LinhNK\Downloads\reviews_reviews_com.citybay.farming.citybuilding_202205.xlsx")
df_6=pd.read_excel(r"C:\Users\iKame_LinhNK\Downloads\reviews_reviews_com.citybay.farming.citybuilding_202206.xlsx")
df_7=pd.read_excel(r"C:\Users\iKame_LinhNK\Downloads\reviews_reviews_com.citybay.farming.citybuilding_202207.xlsx")

In [3]:
df_all=pd.concat([df_5,df_6,df_7])

In [5]:
df_all.columns

Index(['Package Name', 'App Version Code', 'App Version Name',
       'Reviewer Language', 'Device', 'Review Submit Date and Time',
       'Review Submit Millis Since Epoch', 'Review Last Update Date and Time',
       'Review Last Update Millis Since Epoch', 'Star Rating', 'Review Title',
       'Review Text', 'Developer Reply Date and Time',
       'Developer Reply Millis Since Epoch', 'Developer Reply Text',
       'Review Link'],
      dtype='object')

### Translate 
- All non-en reviews will be translated to en 


In [ ]:
def translate(text, source):
    translated_text=''
    if text:
        if source.lower()=='en':
            translated_text=text 
        else: 
            translator = Translator()
            try:
                translated_text = translator.translate(text, src=source).text
            except:
                detected_lang=translator.detect(text).lang
                translated_text = translator.translate(text, src=detected_lang).text 
    return translated_text

In [22]:
translator = Translator()
translator.detect(text='Es divertido').lang

'es'

In [26]:
df_all['translated_text']=df_all.apply(lambda row: translate(text=row['Review Text'], source=row['Reviewer Language']), axis = 1)

In [30]:
def extract_text(text):
    try:
        return text.text
    except:
        return text

In [31]:
df_all['final_translated_text']=df_all.translated_text.apply(lambda x: extract_text(x))

In [36]:
df_all['final_translated_text']=df_all.final_translated_text.apply(lambda x: str(x).lower() if x else str(x))

In [38]:
df_all[(df_all['Reviewer Language'] != 'en') & (~df_all.translated_text.isna())].head()

,Package Name,App Version Code,App Version Name,Reviewer Language,Device,Review Submit Date and Time,Review Submit Millis Since Epoch,Review Last Update Date and Time,Review Last Update Millis Since Epoch,Star Rating,Review Title,Review Text,Developer Reply Date and Time,Developer Reply Millis Since Epoch,Developer Reply Text,Review Link,translated_text,final_translated_text
0,com.citybay.farming.citybuilding,NaN,NaN,id,1904.0,2022-05-01T00:00:24Z,1.651363e+12,2022-05-01T00:00:24Z,1.651363e+12,5.0,NaN,NaN,NaN,NaN,NaN,NaN,"Translated(src=id, dest=en, text=nan, pronunci...",nan
1,com.citybay.farming.citybuilding,197.0,2.8.44,es,ginkgo,2022-05-01T00:20:11Z,1.651364e+12,2022-05-01T00:20:11Z,1.651364e+12,4.0,NaN,Es divertido,NaN,NaN,NaN,http://play.google.com/console/developers/4764...,"Translated(src=es, dest=en, text=It's fun, pro...",it's fun
3,com.citybay.farming.citybuilding,197.0,2.8.44,pt,beyond2,2022-05-01T00:40:28Z,1.651366e+12,2022-05-01T00:40:28Z,1.651366e+12,5.0,NaN,Estou adorando o jogo,NaN,NaN,NaN,http://play.google.com/console/developers/4764...,"Translated(src=pt, dest=en, text=I'm loving th...",i'm loving the game
4,com.citybay.farming.citybuilding,197.0,2.8.44,es,a7y18lte,2022-05-01T00:41:23Z,1.651366e+12,2022-05-01T00:41:23Z,1.651366e+12,5.0,NaN,Es un excelente juego,NaN,NaN,NaN,http://play.google.com/console/developers/4764...,"Translated(src=es, dest=en, text=It's an excel...",it's an excellent game
5,com.citybay.farming.citybuilding,197.0,2.8.44,id,riva,2022-05-01T00:56:34Z,1.651367e+12,2022-05-01T00:56:34Z,1.651367e+12,5.0,NaN,NaN,NaN,NaN,NaN,NaN,"Translated(src=id, dest=en, text=nan, pronunci...",nan


In [68]:
df_all['comment_id']=range(1,19394)

In [87]:
df_all['Review Title'].unique()

array([nan])

### Tokenize  
- Reduce the word to its core 
- Remove punctation

In [169]:
df_comment=df_all[['final_translated_text', 'comment_id']]

In [170]:
df_comment=df_comment[df_comment.final_translated_text!='nan']

In [171]:
len(df_comment)

8084

In [173]:
#remove punctuation 
table = str.maketrans(dict.fromkeys(string.punctuation.replace("'", ''))) 
df_comment['final_translated_text']=df_comment.final_translated_text.apply(lambda x: x.translate(table))

### Remove stopwords

In [175]:
stop_words = set(stopwords.words("english"))
stop_words.remove('not')
def remove_stopwords(sentence):
    filtered_list = []
    ls_words=sentence.split()
    for word in ls_words:
        if word.casefold() not in stop_words:
            filtered_list.append(word)
    return filtered_list

In [176]:
df_comment['tokens_without_stopwords']=df_comment.final_translated_text.apply(lambda x: remove_stopwords(x))

In [177]:
df_comment.head(10)

,final_translated_text,comment_id,tokens_without_stopwords
1,it's fun,2,[fun]
3,i'm loving the game,4,"[i'm, loving, game]"
4,it's an excellent game,5,"[excellent, game]"
6,super cool games,7,"[super, cool, games]"
7,nope,8,[nope]
11,i delete all progress and it does not allow to...,12,"[delete, progress, not, allow, link, fb]"
12,like so much,13,"[like, much]"
16,very good,17,[good]
17,prefer,18,[prefer]
19,just want to try it i'll give it full stars,20,"[want, try, i'll, give, full, stars]"


### Lemmatize Verb 

- Reduce all verbs to V-infinitive 

In [178]:
lemmatizer = WordNetLemmatizer()

In [179]:
#lemmatize verb 
x=['loved', 'addictive', 'game']
[lemmatizer.lemmatize(word, pos="v") for word in x] 

['love', 'addictive', 'game']

In [180]:
df_comment['tokens_lemmatize']=df_comment.tokens_without_stopwords.apply(lambda x: [lemmatizer.lemmatize(word, pos="v") for word in x])

In [181]:
df_comment.head(10)

,final_translated_text,comment_id,tokens_without_stopwords,tokens_lemmatize
1,it's fun,2,[fun],[fun]
3,i'm loving the game,4,"[i'm, loving, game]","[i'm, love, game]"
4,it's an excellent game,5,"[excellent, game]","[excellent, game]"
6,super cool games,7,"[super, cool, games]","[super, cool, game]"
7,nope,8,[nope],[nope]
11,i delete all progress and it does not allow to...,12,"[delete, progress, not, allow, link, fb]","[delete, progress, not, allow, link, fb]"
12,like so much,13,"[like, much]","[like, much]"
16,very good,17,[good],[good]
17,prefer,18,[prefer],[prefer]
19,just want to try it i'll give it full stars,20,"[want, try, i'll, give, full, stars]","[want, try, i'll, give, full, star]"


In [192]:
df_comment_explode=df_comment.explode('tokens_lemmatize')

In [193]:
df_all=df_all.merge(df_comment_explode, on=['comment_id', 'final_translated_text'], how='left')

In [223]:
df_all['total_words']=df_all.final_translated_text.apply(lambda x: len(x.split()) )

In [225]:
df_all.to_csv('Farm_user_review_keywords.csv')